# Download Aquaculture Sites & Export to XLS

This notebook does the following:

1. Download the locations of all aquaculture sites from Barentswatch
2. Keeps only the locations of sites
   1. In Mid-Norway (between 62 and 65 degree of latitude)
   2. That are not on-land
   3. That have salmon
3. Exports the sites to `salmon-sites-midnorway.xlsx`
4. Compute the distance matrix between all sites
5. Exports the distance matrix to `sites-atsea-salmonoids-midnor-distances.xlsx`

To run the notebook, you need API credentials for [Barentswatch](https://www.barentswatch.no).

See also https://developer.barentswatch.no/docs/tutorial.

In [1]:
import logging
from datetime import datetime

import pandas as pd
import pyproj
import requests
from tqdm import tqdm

# Connect to Barentswatch API

In [2]:
BW_CLIENT_ID=
BW_CLIENT_SECRET=

In [3]:
def _get_bw_token():
    """Get time-limited token for API access using client id and password."""
    data = {
        "client_id": BW_CLIENT_ID,
        "scope": "api",
        "client_secret": BW_CLIENT_SECRET,
        "grant_type": "client_credentials",
    }
    response = requests.post("https://id.barentswatch.no/connect/token", data=data)
    if response.status_code == 200:
        token = response.json()["access_token"]
        return token
    else:
        logging.critical(response.json())

token = _get_bw_token()

# Get Info for Aquaculture Sites

## Get Sites

In [4]:
def _get_sites_info(week, year, token):
    """Get basic info on all sites for given year and week."""
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(
        f"https://www.barentswatch.no/bwapi/v1/geodata/fishhealth/locality/{year}/{week}",
        headers=headers,
    )
    if response.status_code == 200:
        df_sites_info = (
            pd.DataFrame(response.json()["localities"])
            .sort_values("name")
            .set_index("localityNo")
        )
        return df_sites_info
    else:
        logging.critical(response.json())

year_now, week_now, _ = datetime.now().isocalendar()
df_sites_info = _get_sites_info(week_now, year_now, token)
df_sites_info

,localityWeekId,name,hasReportedLice,isFallow,avgAdultFemaleLice,hasCleanerfishDeployed,hasMechanicalRemoval,hasSubstanceTreatments,hasPd,hasIla,municipalityNo,municipality,lat,lon,isOnLand,inFilteredSelection,hasSalmonoids,isSlaughterHoldingCage
localityNo,,,,,,,,,,,,,,,,,,
14746,1880823,Aarsand,False,True,NaN,False,False,False,False,False,1811,Bindal,65.045867,12.156933,False,True,False,False
31937,1881193,Abelsnes,False,True,NaN,False,False,False,False,False,4207,Flekkefjord,58.238767,6.656650,True,True,False,False
45119,1881475,Abelsnes II,False,True,NaN,False,False,False,False,False,4207,Flekkefjord,58.240267,6.657250,True,True,False,False
10665,1880210,Adamselv,False,True,NaN,False,False,False,False,False,5624,Lebesby,70.414500,26.702000,True,True,True,False
29196,1881145,Adjetjohka,False,True,NaN,False,False,False,False,False,5612,Guovdageaidnu - Kautokeino,68.944137,22.918715,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,1879067,Øvergården,False,True,NaN,False,False,False,False,False,5503,Harstad,68.981667,16.534667,False,True,True,False
19975,1880963,Øya,False,True,NaN,False,False,False,False,False,1838,GildeskÅL,67.034950,14.018050,True,True,True,False
13153,1880649,Øye,False,True,NaN,False,False,False,False,False,4227,Kvinesdal,58.277059,6.887325,True,True,False,False


## Do Some Filtering (Mid-Norway, On-Land, Salmonoids)

In [5]:
mask = ~df_sites_info['isOnLand'] & df_sites_info['hasSalmonoids']
mask = mask & (df_sites_info['lat'] < 65) & (df_sites_info['lat'] > 62)
df_sites_info_midnor = df_sites_info[mask]
df_sites_info_midnor

,localityWeekId,name,hasReportedLice,isFallow,avgAdultFemaleLice,hasCleanerfishDeployed,hasMechanicalRemoval,hasSubstanceTreatments,hasPd,hasIla,municipalityNo,municipality,lat,lon,isOnLand,inFilteredSelection,hasSalmonoids,isSlaughterHoldingCage
localityNo,,,,,,,,,,,,,,,,,,
13271,1879359,Almurden,False,False,NaN,False,False,False,False,False,5049,Flatanger,64.433533,10.678883,False,True,True,False
32277,1879832,Andholmen 1,True,False,0.072222,False,False,False,False,False,1573,SmØLa,63.469217,7.852300,False,True,True,False
45087,1880079,Aukan,False,False,NaN,False,False,False,False,False,1554,AverØY,63.060300,7.594283,False,True,True,False
12988,1879313,Aukrasanden,False,False,NaN,False,False,False,False,False,1547,Aukra,62.783417,6.926050,False,True,True,False
12357,1879241,Aursøysva,False,False,NaN,False,False,False,True,False,5014,FrØYa,63.801967,8.910767,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24197,1879630,Værøya Ø,False,True,NaN,False,False,False,False,False,5056,Hitra,63.415967,8.483883,False,True,True,False
45032,1880061,Ytterflesa,False,False,NaN,False,False,False,False,False,5060,NÆRØYsund,64.972983,11.443517,False,True,True,False
24695,1879640,Årnes,False,True,NaN,False,False,False,False,False,5007,Namsos,64.596367,11.272917,False,True,True,False


# Calculate Distances Between Sites

In [6]:
def _calculate_distances(df_sites):
    """Calculate the distance matrix for aquaculture sites."""
    df_sites_tmp = df_sites.reset_index()
    site_ids = df_sites_tmp.localityNo
    df_dists = pd.DataFrame(index=site_ids, columns=site_ids)
    geod = pyproj.Geod(ellps="WGS84")
    for site_id in tqdm(site_ids):
        lon_ = float(df_sites_tmp[df_sites_tmp.localityNo == site_id].lon.iloc[0])
        lat_ = float(df_sites_tmp[df_sites_tmp.localityNo == site_id].lat.iloc[0])
        df_dists.loc[site_id] = geod.inv(
            [lon_] * df_sites_tmp.shape[0],
            [lat_] * df_sites_tmp.shape[0],
            df_sites_tmp["lon"].values,
            df_sites_tmp["lat"].values,
        )[2]
    return df_dists

df_distances = _calculate_distances(df_sites_info_midnor)
df_distances

100%|█████████████████████████████████████████████████████████████████████████████████████| 240/240 [00:00<00:00, 2636.81it/s]


localityNo,13271,32277,45087,12988,12357,26795,13572,31877,13845,12625,...,12839,13246,45135,17495,45108,24197,45032,24695,12394,24696
localityNo,,,,,,,,,,,,,,,,,,,,,
13271,0.0,175336.105634,215890.417841,261652.653672,111255.785752,1865.301714,125219.774334,33932.769477,370456.635303,20887.123059,...,293635.139834,348528.161496,357247.944478,196409.71991,191241.54988,156397.124607,70335.748987,33816.520469,132160.27264,132171.88363
32277,175336.105634,0.0,47385.73556,89593.131012,64252.279198,175014.560517,64874.727957,209255.371289,196207.930515,194892.972921,...,135127.021835,175490.699106,182987.220943,50211.333836,19200.062775,32072.511421,241792.828791,209090.417437,43920.880528,43816.313238
45087,215890.417841,47385.73556,0.0,45881.515174,105610.030504,215338.395442,94082.65401,249690.195202,155091.087499,236170.006966,...,87861.234133,132675.007251,141955.616293,31477.28156,28189.120127,59754.894281,284325.598018,249167.211695,89007.854795,88802.257049
12988,261652.653672,89593.131012,45881.515174,0.0,150992.04867,261124.206443,139724.595112,295486.072982,109235.910946,281853.468059,...,58430.28992,86878.650574,96125.049729,71963.345049,71417.284694,105636.408796,329792.302777,294993.367167,133072.995001,132916.220966
12357,111255.785752,64252.279198,105610.030504,150992.04867,0.0,110878.059416,32079.043091,145188.45202,259378.905137,131099.330322,...,188091.141215,237807.108944,246159.646384,90747.083386,80069.549533,47957.810783,178800.591514,144942.962366,23390.283436,23216.563612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24197,156397.124607,32072.511421,59754.894281,105636.408796,47957.810783,155793.879925,35571.622471,190109.898881,214828.815394,176828.623191,...,140134.49105,192379.382003,201679.258033,42991.658524,38723.919622,0.0,225365.43258,189525.50657,38605.642497,38216.820879
45032,70335.748987,241792.828791,284325.598018,329792.302777,178800.591514,71363.604003,195257.033305,40456.03065,437908.260356,49451.931364,...,363656.841496,416586.430433,424683.500201,266179.777015,258696.628453,225365.43258,0.0,42765.220756,197933.518286,198014.729678
24695,33816.520469,209090.417437,249167.211695,294993.367167,144942.962366,34075.882745,157457.649029,3289.114677,403937.701854,17464.52225,...,325553.888735,381841.469577,390738.126546,228824.677176,224833.742515,189525.50657,42765.220756,0.0,165974.40295,165984.616847


# Export

In [7]:
df_sites_info_midnor.to_excel('salmon-sites-midnorway.xlsx')

In [8]:
df_distances.to_excel('sites-atsea-salmonoids-midnor-distances.xlsx')